# `TensorFlow version related error`

In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print('Version:', tf.__version__)
print('Eager mode:', tf.executing_eagerly())
print('Hub version:', hub.__version__)
print('GPU is', 'available' if tf.config.list_physical_devices('GPU') else 'NOT AVAILABLE')

2022-11-23 14:58:54.448268: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-23 14:58:54.448282: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Version: 2.8.0-rc1
Eager mode: True
Hub version: 0.12.0
GPU is NOT AVAILABLE


2022-11-23 14:58:55.469863: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-23 14:58:55.469879: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-23 14:58:55.469891: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nsl54): /proc/driver/nvidia/version does not exist


## Download IMDB dataset

In [2]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews",
                                                  split=('train[:60%]', 'train[60%:]', 'test'),
                                                  as_supervised=True)

2022-11-23 14:58:57.968880: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Explore the data


In [3]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

2022-11-23 15:00:37.290812: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [5]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## Build model
The neural network is created by stacking layers—this requires three main architectural decisions:
- How to represent the text?
- How many layers to use in the model?
- How many hidden units to use for each layer?
In this example, the input data consists of sentences. The labels to predict are either 0 or 1.
<br><br>
One way to represent the text is to convert sentences into embeddings vectors. Use a pre-trained text embedding as the first layer, which will have three advantages:
- We don't have to worry about text preprocessing,
- Benefit from transfer learning,
- the embedding has a fixed size, so it's simpler to process.
For this example we use a pre-trained text embedding model from TensorFlow Hub called **google/nnlm-en-dim50/2**.
<br><br>
There are many other pre-trained text embeddings from TFHub that can be used:
- **google/nnlm-en-dim128/2** - trained with the same NNLM architecture on the same data as `google/nnlm-en-dim50/2`, but with a larger embedding dimension. Larger dimensional embeddings can improve on your task but it may take longer to train our model.
- **google/nnlm-en-dim128-with-normalization/2** - the same as `google/nnlm-en-dim128/2`, but with additional text normalization such as removing punctuation. This can help if the text in your task contains additional characters or punctuation.
- **google/universal-sentence-encoder/4** - a much larger model yielding 512 dimensional embeddings trained with a deep averaging network (DAN) encoder.
<br><br>
Create a Keras layer that uses a TensorFlow Hub model to embed the sentences, and try it out on a couple of input examples. No matter the length of the input text, the output shape of the embeddings is: `(num_examples, embedding_dimension)`.

In [6]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                          dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


The layers are stacked sequentially to build the classifier:
1. The first layer is a TensorFlow Hub layer. This layer uses a pre-trained Saved Model to map a sentence into its embedding vector. The pre-trained text embedding model that you are using (google/nnlm-en-dim50/2) splits the sentence into tokens, embeds each token and then combines the embedding. The resulting dimensions are: `(num_examples, embedding_dimension)`. For this NNLM model, the `embedding_dimension` is 50.
2. This fixed-length output vector is piped through a fully-connected (`Dense`) layer with 16 hidden units.
3. The last layer is densely connected with a single output node.

## Loss function and optimizer
A model needs a loss function and an optimizer for training. Since this is a binary classification problem and the model outputs logits (a single-unit layer with a linear activation), you'll use the `binary_crossentropy` loss function which is better for dealing with probabilities—it measures the "distance" between probability distributions, or in our case, between the ground-truth distribution and the predictions.

In [8]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])

## Train
Train the model for 10 epochs in mini-batches of 512 samples. This is 10 iterations over all samples in the `x_train` and `y_train` tensors. While training, monitor the model's loss and accuracy on the 10,000 samples from the validation set:

In [9]:
history = model.fit(train_data.shuffle(10000).batch(512),
                   epochs=10,
                   validation_data=validation_data.batch(512),
                   verbose=1)

Epoch 1/10
30/30 [==============================] - 13s 412ms/step - loss: 0.6675 - accuracy: 0.5119 - val_loss: 0.6322 - val_accuracy: 0.5450
Epoch 2/10
30/30 [==============================] - 12s 407ms/step - loss: 0.5807 - accuracy: 0.6341 - val_loss: 0.5430 - val_accuracy: 0.7004
Epoch 3/10
30/30 [==============================] - 12s 405ms/step - loss: 0.4649 - accuracy: 0.7776 - val_loss: 0.4430 - val_accuracy: 0.7912
Epoch 4/10
30/30 [==============================] - 12s 405ms/step - loss: 0.3425 - accuracy: 0.8629 - val_loss: 0.3652 - val_accuracy: 0.8350
Epoch 5/10
30/30 [==============================] - 12s 402ms/step - loss: 0.2481 - accuracy: 0.9075 - val_loss: 0.3273 - val_accuracy: 0.8567
Epoch 6/10
30/30 [==============================] - 12s 403ms/step - loss: 0.1839 - accuracy: 0.9375 - val_loss: 0.3126 - val_accuracy: 0.8563
Epoch 7/10
30/30 [==============================] - 12s 404ms/step - loss: 0.1375 - accuracy: 0.9583 - val_loss: 0.3090 - val_accuracy: 0.8600

## Evaluate


In [12]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.3449 - accuracy: 0.8530 - 3s/epoch - 69ms/step
loss: 0.345
accuracy: 0.853


In [17]:
examples = [
    #bad
    "The MCU’s mechanics are too oppressive to allow for true mournful meditation.",
    "I find I respect Coogler’s sequel more than I actually enjoyed it, and that’s as disappointing a turn of events as any I could have imagined.",
    "Feels overstuffed, filled with interesting ideas and possibilities that never fully come into focus.",
    #good
    "Overall, the film is solid with a few passages worth going to the theater for.",
    "may not be the perfect Marvel movie, but [it] prove its worthiness as a bridge – not merely between Phase 4 and Phase 5... but from the bleakness of loss to a stable place from which a new generation can move forward.",
    "Wow! You’ll say that over and over again as this mind-blowing, superhero epic unfolds. Wow!"
    
]

model.predict(examples)

array([[-1.6042079 ],
       [-0.45552784],
       [-1.5944718 ],
       [ 1.6956097 ],
       [-0.7213136 ],
       [-0.3667919 ]], dtype=float32)

In [24]:
from tensorflow.keras import layers, losses

export_model = tf.keras.Sequential([
    model,
    layers.Activation('sigmoid')
])

export_model.compile(loss=losses.BinaryCrossentropy(from_logits=False), optimizer='adam', metrics=['accuracy'])

loss, accuracy = export_model.evaluate(test_data.batch(512))
accuracy

49/49 [==============================] - 4s 70ms/step - loss: 0.3449 - accuracy: 0.8604


0.8604000210762024

In [25]:
export_model.predict(examples)

array([[0.16739428],
       [0.3880473 ],
       [0.16875565],
       [0.84496045],
       [0.3271038 ],
       [0.40931645]], dtype=float32)